In [1]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table
import base64
from dash.dependencies import Input, Output
from bson.json_util import dumps

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pymongo import MongoClient


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from CRUD import AnimalShelter




###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name

username = "aacUser"
password = "password"
shelter = AnimalShelter()
shelter._init_(username,password)


# class read method must support return of cursor object and accept projection json input
df = pd.DataFrame.from_records(shelter.read({}))



#########################
# Dashboard Layout / View
#########################

app = JupyterDash('SimpleExample')

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'Img.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))


app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), style={'height':'15%','width':'15%'}),
    html.Center(html.B(html.H1('Jeffrey Brondell-SNHU CS-340'))),
    html.Hr(),
    dcc.Dropdown(
        id='dropdown',
        options=[
            {'label': 'Water Rescue', 'value': "wtr"},
            {'label': 'Mountain Rescue', 'value': "mtn"},
            {'label': 'Disaster Rescue', 'value': "dstr"},
            {'label': 'Reset', 'value': "rst"}
        ],
        #multi=True,
        placeholder="select a filter option."
    ),
    html.Div(id='dd_test'),
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        #FIXME: Set up the features for your interactive data table to make it user-friendly for your client
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable="single",
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current=0,
        page_size=5,
        

    ),
     html.Br(),
     html.Hr(),
     #html.Center(html.B(html.H1('Jeffrey Brondell-SNHU CS-340'))),
     html.Hr(),
     html.Div(className='row',
         style={'display' : 'flex'},
             children=[
            html.Div(
            id='graph-id',
            className='col s12 m6'),
            html.Div(
            id='map-id',
            className='col s12 m6')
             ]),

])



#############################################
# Interaction Between Components / Controller
#############################################
#This callback will highlight a row on the data table when the user selects it
@app.callback(
    Output('dd_test','children'),
    [Input('dropdown','value')]
)
def update_output(value):
    return 'you have selected "{}"'.format(value)


#callback for interactive filter option via use of dropdown with customized objects for each type of dog (Based on requirements)
@app.callback(
    Output('datatable-id', 'data'),
    [Input('dropdown','value')]
)
def update_dataTable(value):
  
    wtr = {'breed': {"$in": ['Labrador Retriever Mix','Chesapeake Bay Retriever', 'Newfoundland Mix']}, 'sex_upon_outcome': 'Intact Female','age_upon_outcome_in_weeks': {"$gt":26, "$lt": 156}}
    mtn = {'breed': {"$in": ['German Shepherd','Alaskan Malamute','Old English Sheepdog','Siberian Husky','Rottweiler']}, 'sex_upon_outcome': 'Intact Male','age_upon_outcome_in_weeks': {"$gt":26, "$lt": 156}}
    dstr = {'breed': {"$in": ['Doberman Pinscher','German Shepherd', 'Golden  Retriever', 'Bloodhound', 'Rottweiler']}, 'sex_upon_outcome': 'Intact Male','age_upon_outcome_in_weeks': {"$gt":20, "$lt": 300}}
    dog = {'animal_type': 'Dog'}
    
    if value is None:
        df = pd.DataFrame.from_records(shelter.read(dog))
    elif 'wtr' in value:
        df = pd.DataFrame.from_records(shelter.read(wtr))
    elif 'mtn' in value:
        df = pd.DataFrame.from_records(shelter.read(mtn))
    elif 'dstr' in value:
        df = pd.DataFrame.from_records(shelter.read(dstr))
    elif 'rst' in value:
        df = pd.DataFrame.from_records(shelter.read({dog}))
    else:
        df = pd.DataFrame.from_records(shelter.read({dog}))
    
    return df.to_dict('records')

#callback for graph to show virtual (all pages)
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    ###FIX ME ####
    df = pd.DataFrame.from_records(viewData)
    val1 = df.count(axis='columns')/16
    pie_chart = px.pie(df,values=val1,names="breed")
    # add code for chart of your choice (e.g. pie chart) #
    return[
        dcc.Graph(
        figure = pie_chart
        )
    ]
    

#call back receives datatable data and current selection data to display lon and lat of location in geolocation
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', 'derived_viewport_data'),
     Input('datatable-id', 'derived_viewport_selected_rows'),
     Input('datatable-id', 'selected_row_ids'),
    ])
def update_map(viewData, selected_row_ids, derived_viewport_selected_rows):
#FIXME Add in the code for your geolocation chart
    
    dff = pd.DataFrame.from_dict(viewData)
    
    #if no row is selected use the full database, otherwise use only the selected row for info
    if derived_viewport_selected_rows is None:
        derived_viewport_selected_rows = []
        dffs = dff
    else:
        dffs = dff.loc[[selected_row_ids[0]]]
        lat = dffs.iloc[0,13]
        lon = dffs.iloc[0,14]
    
    #returns geolocation of lat and lon of animal selected via datatable, as well as name and breed
    return[
        dl.Map(style={'width':'1000px', 'height':'500px'}, center=[lat,lon], zoom=10, children = [
            dl.TileLayer(id="base-layer-id"),
            #Maker with tool tip and popup
            dl.Marker(position=[lat,lon], children=[
                dl.Tooltip(dffs.iloc[0,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dffs.iloc[0,9])
                ])
            ])
        ])
        
         
    ]
    

app